In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
# from sklearn.metrics import f1_score
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory
import seaborn as sns
import matplotlib.pyplot as plt
import os
import cv2
import glob
print(os.listdir("../input"))


In [ ]:
np.random.seed(42)

In [ ]:
from pathlib import Path
from fastai import *
from fastai.vision import *
import torch
from fastai.callbacks.hooks import *

In [ ]:
## set the data folder
data_folder = Path("../input")

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import numpy as np 
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os
import cv2
import glob

In [ ]:
from pathlib import Path
from fastai import *
from fastai.vision import *
import torch
from fastai.callbacks.hooks import *

In [ ]:
cd /kaggle/input/he_challenge_data/data

In [ ]:
ls

In [ ]:
train=pd.read_csv("trainfinal.csv")
test=pd.read_csv("testfinal.csv")
train.head(), test.head()

In [ ]:
del test['category']

In [ ]:
train['image_id']= train['image_id'].astype(str) + '.jpg'
train.head(5)

In [ ]:
test['image_id']= test['image_id'].astype(str) + '.jpg'
test.head(5)

In [ ]:
submission = pd.DataFrame({'image_id': train['image_id'], 'category': train['category']})
submission.head(10)

In [ ]:
#submission.to_csv('trainfinal.csv', index=False)
train.to_csv("../input/HE_Challenge_data/data/train.csv", index= False)

In [ ]:
path = "/kaggle/input/he_challenge_data/data"

In [ ]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [ ]:
!curl -s https://course.fast.ai/setup/colab | bash

In [ ]:
!pip install https://github.com/fastai/fastai/archive/1.0.49.zip

In [ ]:
from fastai.vision import *
from fastai.callbacks import *

In [ ]:
!pip install fastai

In [ ]:
cd /kaggle/input/he_challenge_data/data

In [ ]:
sys.setrecursionlimit(50000)


In [ ]:
data = ImageDataBunch.from_csv(path=path, folder='train', csv_labels='trainfinal.csv', test='test', ds_tfms=get_transforms(), size=128, bs=128).normalize(imagenet_stats)

In [ ]:

get_transforms()

In [ ]:
data.show_batch()

In [ ]:
print(data.classes)
len(data.classes),data.c

In [ ]:
ls

In [ ]:
learn = create_cnn(data, models.resnet34, metrics=[error_rate, accuracy], model_dir="/tmp/model/")


In [ ]:
learn.fit_one_cycle(4)

In [ ]:
learn.save('stage-1')

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)

losses,idxs = interp.top_losses()

len(data.valid_ds)==len(losses)==len(idxs)

In [ ]:
interp.plot_top_losses(9, figsize=(15,11))

In [ ]:
interp.plot_confusion_matrix(figsize=(12,12), dpi=60)

In [ ]:
interp.most_confused(min_val=10)

In [ ]:
learn.unfreeze()

In [ ]:
learn.fit_one_cycle(1)

In [ ]:
learn.load('stage-1');

In [ ]:
learn.lr_find()

In [ ]:
learn.recorder.plot()

In [ ]:
learn.load('stage-1');
learn.unfreeze()
learn.fit_one_cycle(2)

In [ ]:
learn50 = create_cnn(data, models.resnet50, metrics=error_rate, model_dir="/tmp/model/")

In [ ]:
learn50.lr_find()
learn50.recorder.plot()

In [ ]:
learn50.fit_one_cycle(8)

In [ ]:
learn50.save('stage-1-50')

In [ ]:
learn50.load('stage-1-50')
learn50.unfreeze()
learn50.fit_one_cycle(5, max_lr=slice(1e-4,1e-2))

In [ ]:
log_preds, test_labels = learn50.get_preds(ds_type=DatasetType.Test)

In [ ]:
preds = np.argmax(log_preds, 1)
preds

In [ ]:
import numpy as np
import pandas as pd

In [ ]:

a = np.array(preds)
a

In [ ]:
a=a+1

In [ ]:
a.min()

In [ ]:
print(data.test_ds.x[0])

In [ ]:
df = pd.DataFrame({'image_id':test['image_id'], 'category':a}, columns=['image', 'category'])
df.to_csv('submission.csv', index=False)

In [ ]:
from IPython.display import HTML
def create_download_link(title = "Download CSV file", filename = "data.csv"):  
    html = '<a href={filename}>{title}</a>'
    html = html.format(title=title,filename=filename)
    return HTML(html)

In [ ]:
create_download_link(filename = 'submission.csv')

In [ ]:
sizes = [32, 64, 128, 224]

In [ ]:
def get_data(sz, bs):
  data = ImageDataBunch.from_csv(path=path, folder='train', csv_labels='trainfinal.csv', test='test', ds_tfms=get_transforms(), size=sz, bs=bs).normalize(imagenet_stats)
  return data

In [ ]:
learn50 = create_cnn(get_data(8, int(2048/8)), models.resnet50, metrics=error_rate, model_dir="/tmp/model/")
learn50.save('res50_0')
learn50.save('res50_8')
learn50 = create_cnn(get_data(16, int(2048/16)), models.resnet50, metrics=error_rate, model_dir="/tmp/model/").load('res50_8')
learn50.save('res50_16')
learn50 = create_cnn(get_data(24, int(2048/24)), models.resnet50, metrics=error_rate, model_dir="/tmp/model/").load('res50_16')
learn50.save('res50_24')
learn50 = create_cnn(get_data(32, int(2048/32)), models.resnet50, metrics=error_rate, model_dir="/tmp/model/").load('res50_24')
learn50.save('res50_32')
learn50 = create_cnn(get_data(64, int(2048/64)), models.resnet50, metrics=error_rate, model_dir="/tmp/model/").load('res50_32')
learn50.save('res50_64')
learn50 = create_cnn(get_data(128, int(2048/128)), models.resnet50, metrics=error_rate, model_dir="/tmp/model/").load('res50_64')
learn50.save('res50_128')
learn50 = create_cnn(get_data(224, int(2048/224)), models.resnet50, metrics=error_rate, model_dir="/tmp/model/").load('res50_128')
learn50.save('res50_224')

In [ ]:
def train_model(sz, i):
  learn50 = cnn_learner(get_data(sz, int(2048/sz)), models.resnet50, metrics=[error_rate, accuracy], model_dir="/tmp/model/").load('res50_'+str(sz-8))
  learn50.fit_one_cycle(6*i)
  learn50.lr_find()
  learn50.recorder.plot()
  learn50.unfreeze()
  learn50.fit_one_cycle(2*i)
  learn50.save('res50_'+str(sz))

In [ ]:
data_path = "/kaggle/input/he_challenge_data/data/train/"
path = os.path.join(data_path , "*jpg")
#path = os.path.join( , "*jpg")

In [ ]:
files = glob.glob(path)
trainimage=[]
for file in files:
    image = cv2.imread(file)
    trainimage.append(image)

In [ ]:
data_path = "/kaggle/input/he_challenge_data/data/test/"
path = os.path.join(data_path , "*jpg")
#path = os.path.join( , "*jpg")

In [ ]:
files = glob.glob(path)
testimage=[]
for file in files:
    image = cv2.imread(file)
    testimage.append(image)